<a href="https://colab.research.google.com/github/txvuong150764/Vietnamese-Synonym-PhoBert/blob/master/Synonym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import Library

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 36.4 MB/s 
     |████████████████████████████████| 182 kB 100.0 MB/s 
     |████████████████████████████████| 7.6 MB 84.9 MB/s 


In [ ]:
from tensorflow import keras
from keras.layers import Dense
import tensorflow as tf
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer, RobertaForMaskedLM
import pandas as pd
import string
import json
import random

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Install vncorenlp

In [ ]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git
!pip install vncorenlp
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP('VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

def word_segment(text):
    word_segmented_text = annotator.tokenize(text)
    sentences = []
    for sentence in word_segmented_text:
        sentences.append(' '.join(sentence))
    return ' '.join(sentences)

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 236 (delta 14), reused 1 (delta 0), pack-reused 212
Receiving objects: 100% (236/236), 214.22 MiB | 17.57 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 29.8 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645950 sha256=40a43655b138d02bed05d4ac81f2ad3d4f7e84b8ceb3813bcbf8d59662193a49
  Stored in directory: /root/.cache/pip/wheels/3c/e9/86/706911c12e307aeb9a2702399f0dad38d36f1d6f9dde8af35e
Successfully built vncorenlp


In [ ]:
word_segment('cao bằng có bao nhiêu dân tộc')

'cao_bằng có bao_nhiêu dân_tộc'

# Question Similarity Model

In [ ]:
class EquivalenceClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EquivalenceClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load Question Similarity Model


In [ ]:
model = EquivalenceClassifier(n_classes=2)
model.to(device)
model.load_state_dict(torch.load(f'/content/drive/MyDrive/phobert_fold6.pth'))

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

# Sentence Similarity Check

In [ ]:
class_names = ['False', 'True']

def infer1(tmp_dict, tokenizer, max_len=128):
    encoded_review = tokenizer(
        tmp_dict['text1'],
        tmp_dict['text2'],
        max_length=max_len,
        truncation=True,
        add_special_tokens=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)

    #print(f'Text1: {text1}')
    #print(f'Text2: {text2}')
    #print(f'Is_equivalence: {class_names[y_pred]}')
    return y_pred

# Load File and Model


In [ ]:
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Viet74K_Tokenized.txt') as f:
    tmp_dictionary = f.readlines()

with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-stopwords-master/vietnamese-stopwords-dash.txt') as f:
    tmp_stop_word = f.readlines()

In [ ]:
dictionary = []
for word in tmp_dictionary:
    word = word.replace('\n', '')
    dictionary.append(word)

In [ ]:
stop_word = []
for word in tmp_stop_word:
    word = word.replace('\n', '')
    stop_word.append(word)

In [ ]:
punctuation_set = set(string.punctuation)

In [ ]:
trec57_train_df = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/trec57/train.csv')
trec57_test_df = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/trec57/test.csv')
trec57_dev_df = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/trec57/dev.csv')

In [ ]:
len(trec57_train_df) + len(trec57_test_df) + len(trec57_dev_df)

In [ ]:
trec57_train_df = trec57_train_df.drop(trec57_train_df[trec57_train_df['label'] == 'CONVERSTATION'].index)
trec57_test_df = trec57_test_df.drop(trec57_test_df[trec57_test_df['label'] == 'CONVERSTATION'].index)
trec57_dev_df = trec57_dev_df.drop(trec57_dev_df[trec57_dev_df['label'] == 'CONVERSTATION'].index)

trec57_train_df = trec57_train_df.drop(trec57_train_df[trec57_train_df['label'] == 'BAD'].index)
trec57_test_df = trec57_test_df.drop(trec57_test_df[trec57_test_df['label'] == 'BAD'].index)
trec57_dev_df = trec57_dev_df.drop(trec57_dev_df[trec57_dev_df['label'] == 'BAD'].index)

In [ ]:
trec57_train_df[trec57_train_df['label'] == 'CONVERSTATION']

In [ ]:
trec57_train_df = trec57_train_df.drop(['label'], axis=1)
trec57_test_df = trec57_test_df.drop(['label'], axis=1)
trec57_dev_df = trec57_dev_df.drop(['label'], axis=1)

In [ ]:
li = [trec57_train_df, trec57_test_df, trec57_dev_df]

In [ ]:
trec57_question_list = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
trec57_question_list

In [ ]:
wiki_question_list = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/wiki75/wiki74-revising.csv')

In [ ]:
wiki_question_list['label'].unique()

In [ ]:
wiki_question_list = wiki_question_list.drop(['g_id', 'id', 'label', 'set'], axis=1)

In [ ]:
import json
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/QAC/version/v0.0.1/train.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

qac_question_list = []

for json_str in json_list:
    result = json.loads(json_str)
    if result['question'] not in qac_question_list:
      qac_question_list.append(result['question'])

In [ ]:
import json
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/QAC/version/v0.0.1/test.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    if result['question'] not in qac_question_list:
      qac_question_list.append(result['question'])

In [ ]:
import json
with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/QAC/version/v0.0.1/dev.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    if result['question'] not in qac_question_list:
      qac_question_list.append(result['question'])

In [ ]:
len(qac_question_list)

In [ ]:
all_question_list = []

# for question in trec57_question_list['text']:
#   all_question_list.append(question)

for question in wiki_question_list['text']:
  all_question_list.append(question)

for question in qac_question_list:
  all_question_list.append(question)

In [ ]:
for question in all_question_list:
  if type(question) != str:
    all_question_list.remove(question)

In [ ]:
all_tokenzied_question_list = []
for i in tqdm(range(len(all_question_list))):
  all_tokenzied_question_list.append(word_segment(all_question_list[i]))

In [ ]:
noun_and_question_list = {}
np_and_question_list = {}
nc_and_question_list = {}
p_and_question_list = {}
b_and_question_list = {}
adj_and_question_list = {}
verb_and_question_list = {}

for question in tqdm(all_question_list):
  pos_tag = annotator.pos_tag(question)
  for i in range(len(pos_tag[0])):
    tag = pos_tag[0][i][1]
    if tag == 'N':
      word = pos_tag[0][i][0]
      word_ner = annotator.ner(word.replace('_', ' '))
      if word_ner[0][0][1] != 'B-LOC' and word_ner[0][0][1] != 'B-PER' and word_ner[0][0][1] != 'B-ORG':
        if word in noun_and_question_list.keys():
          noun_and_question_list[word].append(word_segment(question))
        else:
          noun_and_question_list[word] = [word_segment(question)]

    if tag == 'A':
      word = pos_tag[0][i][0]
      if word in adj_and_question_list.keys():
        adj_and_question_list[word].append(word_segment(question))
      else:
        adj_and_question_list[word] = [word_segment(question)]

    if tag == 'V':
      word = pos_tag[0][i][0]
      if word in verb_and_question_list.keys():
        verb_and_question_list[word].append(word_segment(question))
      else:
        verb_and_question_list[word] = [word_segment(question)]

    if tag == 'Np':
      word = pos_tag[0][i][0]
      if word in np_and_question_list.keys():
        np_and_question_list[word].append(word_segment(question))
      else:
        np_and_question_list[word] = [word_segment(question)]

    if tag == 'Nc':
      word = pos_tag[0][i][0]
      if word in nc_and_question_list.keys():
        nc_and_question_list[word].append(word_segment(question))
      else:
        nc_and_question_list[word] = [word_segment(question)]

    if tag == 'P':
      word = pos_tag[0][i][0]
      if word in p_and_question_list.keys():
        p_and_question_list[word].append(word_segment(question))
      else:
        p_and_question_list[word] = [word_segment(question)]

    if tag == 'B':
      word = pos_tag[0][i][0]
      if word in b_and_question_list.keys():
        b_and_question_list[word].append(word_segment(question))
      else:
        b_and_question_list[word] = [word_segment(question)]

In [ ]:
all_word_list = {
    'noun': [],
    'np': [],
    'nc': [],
    'p': [],
    'b': [],
    'adj': [],
    'verb': [],
}

for key in noun_and_question_list.keys():
  all_word_list['noun'].append(key)

for key in np_and_question_list.keys():
  all_word_list['np'].append(key)

for key in nc_and_question_list.keys():
  all_word_list['nc'].append(key)

for key in b_and_question_list.keys():
  all_word_list['b'].append(key)

for key in p_and_question_list.keys():
  all_word_list['p'].append(key)

for key in adj_and_question_list.keys():
  all_word_list['adj'].append(key)

for key in verb_and_question_list.keys():
  all_word_list['verb'].append(key)

In [ ]:
import json
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/all_word_list.json","w", encoding='utf-8') as jsonfile:
    json.dump(all_word_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/noun_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(noun_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/adj_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/verb_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/np_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(np_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/nc_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(nc_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/b_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(b_and_question_list,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/p_and_question.json","w", encoding='utf-8') as jsonfile:
    json.dump(p_and_question_list,jsonfile,ensure_ascii=False, indent=2)

# Function to find Synonym Group

In [ ]:
def listSynonymGroup(data, word_list):
  word = {}
  for key in tqdm(data.keys()):
    detokenized_key = key.replace('_', ' ')
    cnt = 1
    for list_word in word_list:
      if detokenized_key in list_word:
        if detokenized_key in word.keys():
          word[detokenized_key][f'group {cnt}'] = list_word
        else:
          word[detokenized_key] = {f'group {cnt}' : list_word}
        cnt += 1

  return word

# Find all **Nouns** which are both appear in Wordnet and Bert


In [ ]:
all_similar_wordnet = {}

import json
noun_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/noun_and_question.json', encoding='utf-8')
noun_dict = json.loads(noun_file.read())

In [ ]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = '/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-wordnet/Noun'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
noun_df_list = (pd.read_csv(file, names=range(10), skipinitialspace = True, header=None) for file in csv_files)

# Concatenate all DataFrames
big_noun_df   = pd.concat(noun_df_list, ignore_index=True)

In [ ]:
big_noun_df_list = big_noun_df.values.tolist()

In [ ]:
for list_word in big_noun_df_list:
  for i in range(len(list_word)):
    if type(list_word[i]) is float:
      del list_word[i:len(list_word)]
      break

In [ ]:
noun_word = listSynonymGroup(noun_dict, big_noun_df_list)

100%|██████████| 3590/3590 [00:06<00:00, 571.75it/s]


In [ ]:
all_similar_wordnet['noun'] = noun_word

# Find all **Verbs** which are both appear in Wordnet and Bert



In [ ]:
import json
verb_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/verb_and_question.json', encoding='utf-8')
verb_dict = json.loads(verb_file.read())

In [ ]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = '/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-wordnet/Verb'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file, names=range(10), skipinitialspace = True, header=None) for file in csv_files)

# Concatenate all DataFrames
big_verb_df   = pd.concat(df_list, ignore_index=True)

In [ ]:
big_verb_df_list = big_verb_df.values.tolist()

In [ ]:
for list_word in big_verb_df_list:
  for i in range(len(list_word)):
    if type(list_word[i]) is float:
      del list_word[i:len(list_word)]
      break

In [ ]:
verb_word = listSynonymGroup(verb_dict, big_verb_df_list)

100%|██████████| 1933/1933 [00:01<00:00, 1065.05it/s]


In [ ]:
all_similar_wordnet['verb'] = verb_word

# Find all **Adj** which are both appear in Wordnet and Bert



In [ ]:
import json
adj_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/adj_and_question.json', encoding='utf-8')
adj_dict = json.loads(adj_file.read())

In [ ]:
# Import libraries
import glob
import pandas as pd

df_list = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-wordnet/adj.csv', names=range(10), skipinitialspace = True, header=None)

In [ ]:
adj_df_list = df_list.values.tolist()

In [ ]:
for list_word in adj_df_list:
  for i in range(len(list_word)):
    if type(list_word[i]) is float:
      del list_word[i:len(list_word)]
      break

In [ ]:
adj_word = listSynonymGroup(adj_dict, adj_df_list)

100%|██████████| 514/514 [00:00<00:00, 1735.60it/s]


In [ ]:
all_similar_wordnet['adj'] = adj_word

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/all_similar_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(all_similar_wordnet,jsonfile,ensure_ascii=False, indent=2)

# Function to summarize number of Synonym

In [ ]:
def summarizeNumOfSynonym(data):
  cnt_dict = {}
  for key in data.keys():
    max_len = 0
    group_to_removed = []
    synonym_group = {}
    for group in data[key]:
      if (max_len < len(data[key][group]) - 1):
        max_len = len(data[key][group]) - 1

    for group in data[key].keys():
      data[key][group].remove(key)
      if len(data[key][group]) == 0:
        group_to_removed.append(group)

    for group in group_to_removed:
      del data[key][group]

    if max_len in cnt_dict:
      cnt_dict[max_len][key] = data[key]
    else:
      cnt_dict[max_len] = {key:data[key]}

  return cnt_dict

# Summarize number of Synonym in Wordnet

In [ ]:
wordnet_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/all_similar_wordnet.json')
wordnet_data = json.load(wordnet_file)

In [ ]:
noun_data = wordnet_data['noun']
verb_data = wordnet_data['verb']
adj_data = wordnet_data['adj']

**Noun**

In [ ]:
cnt_noun_wordnet = sorted(summarizeNumOfSynonym(noun_data).items(), key=lambda x:x[0], reverse=True)

In [ ]:
cnt_noun_wordnet_dict = {k:v for k,v in cnt_noun_wordnet}

In [ ]:
for key in cnt_noun_wordnet_dict.keys():
  print(f'Number of Noun which has {key} synonym words: {len(cnt_noun_wordnet_dict[key])}')

Number of Noun which has 9 synonym words: 45
Number of Noun which has 8 synonym words: 36
Number of Noun which has 7 synonym words: 73
Number of Noun which has 6 synonym words: 92
Number of Noun which has 5 synonym words: 170
Number of Noun which has 4 synonym words: 209
Number of Noun which has 3 synonym words: 305
Number of Noun which has 2 synonym words: 311
Number of Noun which has 1 synonym words: 306
Number of Noun which has 0 synonym words: 187


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/noun_summarization.json","w", encoding='utf-8') as jsonfile:
    json.dump(cnt_noun_wordnet_dict,jsonfile,ensure_ascii=False, indent=2)

**Verb**

In [ ]:
cnt_verb_wordnet = sorted(summarizeNumOfSynonym(verb_data).items(), key=lambda x:x[0], reverse=True)

In [ ]:
cnt_verb_wordnet_dict = {k:v for k,v in cnt_verb_wordnet}

In [ ]:
for key in cnt_verb_wordnet_dict:
  print(f'Number of Verb which has {key} synonym words: {len(cnt_verb_wordnet_dict[key])}')

Number of Verb which has 9 synonym words: 230
Number of Verb which has 8 synonym words: 78
Number of Verb which has 7 synonym words: 115
Number of Verb which has 6 synonym words: 131
Number of Verb which has 5 synonym words: 128
Number of Verb which has 4 synonym words: 85
Number of Verb which has 3 synonym words: 82
Number of Verb which has 2 synonym words: 77
Number of Verb which has 1 synonym words: 41
Number of Verb which has 0 synonym words: 19


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/verb_summarization.json","w", encoding='utf-8') as jsonfile:
    json.dump(cnt_verb_wordnet_dict,jsonfile,ensure_ascii=False, indent=2)

**Adj**

In [ ]:
cnt_adj_wordnet = sorted(summarizeNumOfSynonym(adj_data).items(), key=lambda x:x[0], reverse=True)

In [ ]:
cnt_adj_wordnet_dict = {k:v for k,v in cnt_adj_wordnet}

In [ ]:
for key in cnt_adj_wordnet_dict:
  print(f'Number of Adj which has {key} synonym words: {len(cnt_adj_wordnet_dict[key])}')

Number of Adj which has 9 synonym words: 45
Number of Adj which has 8 synonym words: 12
Number of Adj which has 7 synonym words: 47
Number of Adj which has 6 synonym words: 37
Number of Adj which has 5 synonym words: 47
Number of Adj which has 4 synonym words: 45
Number of Adj which has 3 synonym words: 41
Number of Adj which has 2 synonym words: 33
Number of Adj which has 1 synonym words: 29
Number of Adj which has 0 synonym words: 9


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/adj_summarization.json","w", encoding='utf-8') as jsonfile:
    json.dump(cnt_adj_wordnet_dict,jsonfile,ensure_ascii=False, indent=2)

# Choose 150 nouns randomly

In [ ]:
import json
noun_candidate_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/noun_summarization.json', encoding='utf-8')
noun_candidate_dict = json.loads(noun_candidate_file.read())

In [ ]:
import random

list_3_6 = []
list_7_9 = []

random_noun = []
for key in noun_candidate_dict.keys():
  if key == '0':
    random_noun.extend(random.sample(list(noun_candidate_dict[key].keys()), 15))
  elif key == '1':
    random_noun.extend(random.sample(list(noun_candidate_dict[key].keys()), 45))
  elif key == '2':
    random_noun.extend(random.sample(list(noun_candidate_dict[key].keys()), 45))
  elif key == '3' or key == '4' or key == '5' or key == '6':
    list_3_6.extend(list(noun_candidate_dict[key].keys()))
  else:
    list_7_9.extend(list(noun_candidate_dict[key].keys()))

random_noun.extend(random.sample(list_3_6, 30))
random_noun.extend(random.sample(list_7_9, 15))

In [ ]:
import collections
print([item for item, count in collections.Counter(random_noun).items() if count > 1])
print('năm' in random_noun)

[]
False


# Choose 100 Verbs randomly


In [ ]:
import json
verb_candidate_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/verb_summarization.json', encoding='utf-8')
verb_candidate_dict = json.loads(verb_candidate_file.read())

In [ ]:
import random

list_3_6 = []
list_7_9 = []

random_verb = []
for key in verb_candidate_dict.keys():
  if key == '0':
    random_verb.extend(random.sample(list(verb_candidate_dict[key].keys()), 10))
  elif key == '1':
    random_verb.extend(random.sample(list(verb_candidate_dict[key].keys()), 30))
  elif key == '2':
    random_verb.extend(random.sample(list(verb_candidate_dict[key].keys()), 30))
  elif key == '3' or key == '4' or key == '5' or key == '6':
    list_3_6.extend(list(verb_candidate_dict[key].keys()))
  else:
    list_7_9.extend(list(verb_candidate_dict[key].keys()))

random_verb.extend(random.sample(list_3_6, 20))
random_verb.extend(random.sample(list_7_9, 10))

In [ ]:
import collections
print([item for item, count in collections.Counter(random_verb).items() if count > 1])

[]


# Choose 50 Adj randomly

In [ ]:
import json
adj_candidate_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/adj_summarization.json', encoding='utf-8')
adj_candidate_dict = json.loads(adj_candidate_file.read())

In [ ]:
import random

list_3_6 = []
list_7_9 = []

random_adj = []
for key in adj_candidate_dict.keys():
  if key == '0':
    print(0)
    random_adj.extend(random.sample(list(adj_candidate_dict[key].keys()), 5))
  elif key == '1':
    print(1)
    random_adj.extend(random.sample(list(adj_candidate_dict[key].keys()), 15))
  elif key == '2':
    print(2)
    random_adj.extend(random.sample(list(adj_candidate_dict[key].keys()), 15))
  elif key == '3' or key == '4' or key == '5' or key == '6':
    print(3)
    list_3_6.extend(list(adj_candidate_dict[key].keys()))
  else:
    print(4)
    list_7_9.extend(list(adj_candidate_dict[key].keys()))

random_adj.extend(random.sample(list_3_6, 10))
random_adj.extend(random.sample(list_7_9, 5))

4
4
4
3
3
3
3
2
1
0


In [ ]:
import collections
print([item for item, count in collections.Counter(random_adj).items() if count > 1])

[]


# All similar word to wordnet

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/all_similar_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(all_similar_wordnet,jsonfile,ensure_ascii=False, indent=2)

# Load 300 words

In [ ]:
noun_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/150_noun_synonym.json')
noun_data = json.load(noun_file)

verb_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/100_verb_synonym.json')
verb_data = json.load(verb_file)

adj_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/50_adj_synonym.json')
adj_data = json.load(adj_file)

In [ ]:
noun_keys = list(noun_data.keys())
verb_keys = list(verb_data.keys())
adj_keys = list(adj_data.keys())

In [ ]:
all_300_words = noun_keys + verb_keys + adj_keys

# List of 300 Words

In [ ]:
final_words_dict = {}

final_words_dict['noun'] = noun_keys
final_words_dict['verb'] = verb_keys
final_words_dict['adj'] = adj_keys

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/300_words_final.json","w", encoding='utf-8') as jsonfile:
    json.dump(final_words_dict,jsonfile,ensure_ascii=False, indent=2)

# Extract synonym set from Wordnet

**Fuction to extract synonym groups in Wordnet**

In [ ]:
def extractSynonymWordnet(keys, df_list):
  res_dict = {}
  cnt = 1
  isIncreased=False
  for key in tqdm(keys):
    group_of_synonym = {}
    detokenized_key = key.replace('_', ' ')
    for list_word in df_list:
      if detokenized_key in list_word and len(list_word) > 1:
        group_of_synonym[f'group {cnt}'] = list_word
        cnt += 1
    res_dict[key] = group_of_synonym
    cnt = 1
  return res_dict

**Synonym for Noun**

In [ ]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = '/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-wordnet/Noun'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_noun_list = (pd.read_csv(file, names=range(10), skipinitialspace = True, header=None) for file in csv_files)

# Concatenate all DataFrames
big_noun_df   = pd.concat(df_noun_list, ignore_index=True)

In [ ]:
def cleanNan(big_df_list):
  for list_word in big_df_list:
    for i in range(len(list_word)):
      if type(list_word[i]) is float:
        del list_word[i:len(list_word)]
        break

  return big_df_list

In [ ]:
big_noun_df_list = big_noun_df.values.tolist()
big_noun_df_list = cleanNan(big_noun_df_list)

In [ ]:
wordnet_noun_synonym = extractSynonymWordnet(noun_keys, big_noun_df_list)

100%|██████████| 150/150 [00:00<00:00, 564.48it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/wordnet_noun_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(wordnet_noun_synonym,jsonfile,ensure_ascii=False, indent=2)

**Synonym for Verb**

In [ ]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = '/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-wordnet/Verb'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_verb_list = (pd.read_csv(file, names=range(10), skipinitialspace = True, header=None) for file in csv_files)

# Concatenate all DataFrames
big_verb_df   = pd.concat(df_verb_list, ignore_index=True)

In [ ]:
big_verb_df_list = big_verb_df.values.tolist()
big_verb_df_list = cleanNan(big_verb_df_list)

In [ ]:
wordnet_verb_synonym = extractSynonymWordnet(verb_keys, big_verb_df_list)

100%|██████████| 100/100 [00:00<00:00, 1137.29it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/wordnet_verb_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(wordnet_verb_synonym,jsonfile,ensure_ascii=False, indent=2)

**Synonym for Adj**

In [ ]:
# Import libraries
import glob
import pandas as pd

df_adj_list = pd.read_csv('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/vietnamese-wordnet/adj.csv', names=range(10), skipinitialspace = True, header=None)

In [ ]:
df_adj_list = df_adj_list.values.tolist()
df_adj_list = cleanNan(df_adj_list)

In [ ]:
wordnet_adj_synonym = extractSynonymWordnet(adj_keys, df_adj_list)

100%|██████████| 50/50 [00:00<00:00, 1467.54it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/wordnet_adj_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(wordnet_adj_synonym,jsonfile,ensure_ascii=False, indent=2)

# Function to find all candidate for Synonym

In [ ]:
def findPosSynonym(data, keys):
  list_of_pos_synonym = {}

  for key in tqdm(keys):
    question_word_dict = {}
    for sentence in data[key]:
      raw_sentence = sentence
      mask_sentence = sentence.replace(key, '<mask>', 1)
      unmasked_result = unmasker(mask_sentence, top_k=50)

      for i in range(50):
        word = unmasked_result[i]['token_str']
        prob = unmasked_result[i]['score']
        if sentence in question_word_dict.keys():
          if word not in question_word_dict[sentence] and word not in punctuation_set and '@' not in word:
            question_word_dict[sentence].append([word, prob])
        else:
          question_word_dict[sentence] = [[word, prob]]

    list_of_pos_synonym[key] = question_word_dict

  return list_of_pos_synonym

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask", 'vinai/phobert-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Find all cadidate for Noun Synonym

In [ ]:
noun_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/noun_and_question.json')
noun_data = json.load(noun_file)

In [ ]:
list_of_pos_noun_synonym = findPosSynonym(noun_data, noun_keys)

100%|██████████| 150/150 [02:46<00:00,  1.11s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Candidate/pos_synonym_150_noun.json","w", encoding='utf-8') as jsonfile:
    json.dump(list_of_pos_noun_synonym,jsonfile,ensure_ascii=False, indent=2)

# Find all cadidate for Verb Synonym


In [ ]:
verb_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/verb_and_question.json')
verb_data = json.load(verb_file)

In [ ]:
list_of_pos_verb_synonym = findPosSynonym(verb_data, verb_keys)

100%|██████████| 100/100 [04:47<00:00,  2.88s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Candidate/pos_synonym_100_verb.json","w", encoding='utf-8') as jsonfile:
    json.dump(list_of_pos_verb_synonym,jsonfile,ensure_ascii=False, indent=2)

# Find all candidate for Adj Synonym

In [ ]:
adj_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/adj_and_question.json')
adj_data = json.load(adj_file)

In [ ]:
list_of_pos_adj_synonym = findPosSynonym(adj_data, adj_keys)

100%|██████████| 50/50 [00:19<00:00,  2.53it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Candidate/pos_synonym_50_adj.json","w", encoding='utf-8') as jsonfile:
    json.dump(list_of_pos_adj_synonym,jsonfile,ensure_ascii=False, indent=2)

# Function to find Synonym

In [ ]:
def findSynonym(pos_synonym_list):
  synonym_dict = {}

  batch_sentence = {
      'text1': [],
      'text2': []
  }

  word_and_prob = {
      'word': [],
      'prob': [],
      'sentence': []
  }

  batch_size = 0
  cnt = 0

  for key in tqdm(pos_synonym_list.keys()):
    small_synonym_dict = {}
    for sub_key in pos_synonym_list[key]:
      for i in range(len(pos_synonym_list[key][sub_key])):
        pos_synonym_word = pos_synonym_list[key][sub_key][i][0]
        if pos_synonym_word in punctuation_set:
          continue

        synonym_score = pos_synonym_list[key][sub_key][i][1]
        sentence = sub_key.replace(key, pos_synonym_word)

        if batch_size < 16:
          batch_sentence['text1'].append(sub_key)
          batch_sentence['text2'].append(sentence)
          word_and_prob['word'].append(pos_synonym_word)
          word_and_prob['prob'].append(synonym_score)
          word_and_prob['sentence'].append(sub_key)

          batch_size += 1
          cnt += 1
        else:
          batch_size = 0
          list_similarity_score = infer1(batch_sentence, tokenizer)
          batch_sentence['text1'] = []
          batch_sentence['text2'] = []

          for k in range(len(list_similarity_score.tolist())):
            similarity_score = list_similarity_score.tolist()[k]
            if similarity_score == 1:
              if word_and_prob['sentence'][k] in small_synonym_dict.keys():
                if word_and_prob['word'][k] not in small_synonym_dict[word_and_prob['sentence'][k]].keys():
                  small_synonym_dict[word_and_prob['sentence'][k]][word_and_prob['word'][k]] = word_and_prob['prob'][k]
              else:
                small_synonym_dict[word_and_prob['sentence'][k]] = {word_and_prob['word'][k]: word_and_prob['prob'][k]}

          word_and_prob['word'] = []
          word_and_prob['prob'] = []
          word_and_prob['sentence'] = []

      if batch_size > 0:
        batch_size = 0
        list_similarity_score = infer1(batch_sentence, tokenizer)
        batch_sentence['text1'] = []
        batch_sentence['text2'] = []

        for k in range(len(list_similarity_score.tolist())):
          similarity_score = list_similarity_score.tolist()[k]
          if similarity_score == 1:
            if word_and_prob['sentence'][k] in small_synonym_dict.keys():
              if word_and_prob['word'][k] not in small_synonym_dict[word_and_prob['sentence'][k]].keys():
                small_synonym_dict[word_and_prob['sentence'][k]][word_and_prob['word'][k]] = word_and_prob['prob'][k]
            else:
              small_synonym_dict[word_and_prob['sentence'][k]] = {word_and_prob['word'][k]: word_and_prob['prob'][k]}

        word_and_prob['word'] = []
        word_and_prob['prob'] = []
        word_and_prob['sentence'] = []
      synonym_dict[key] = small_synonym_dict

  return synonym_dict

# Find Synonym for Noun

In [ ]:
import json
pos_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Candidate/pos_synonym_150_noun.json', encoding='utf-8')
pos_synonym_list = json.loads(pos_synonym_file.read())

In [ ]:
synonym_dict = findSynonym(pos_synonym_list)

100%|██████████| 150/150 [13:16<00:00,  5.31s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/150_noun_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_dict,jsonfile,ensure_ascii=False, indent=2)

# Find Synonym for Verb

In [ ]:
import json
pos_verb_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Candidate/pos_synonym_100_verb.json', encoding='utf-8')
pos_verb_synonym_list = json.loads(pos_verb_synonym_file.read())

In [ ]:
verb_synonym_dict = findSynonym(pos_verb_synonym_list)

100%|██████████| 100/100 [22:35<00:00, 13.56s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/100_verb_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

# Find Synonym for Adj

In [ ]:
import json
pos_adj_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Candidate/pos_synonym_50_adj.json', encoding='utf-8')
pos_adj_synonym_list = json.loads(pos_adj_synonym_file.read())

In [ ]:
adj_synonym_dict = findSynonym(pos_adj_synonym_list)

100%|██████████| 50/50 [01:39<00:00,  1.98s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/50_adj_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

# Function to count Word Occurrence

In [ ]:
def wordOccurrence(synonym_dict):
  synonym_word = {}

  for key in tqdm(synonym_dict.keys()):
    word_occurence = {}
    for sub_key in synonym_dict[key]:
      for word in synonym_dict[key][sub_key]:
        if word in word_occurence:
          word_occurence[word] += 1
        else:
          word_occurence[word] = 1
    word_occurence = {k: v for k, v in sorted(word_occurence.items(), key=lambda item: item[1], reverse=True)}
    synonym_word[key] = word_occurence

  return synonym_word

# Word Occurrence for Noun

In [ ]:
import json
synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/150_noun_synonym.json', encoding='utf-8')
synonym_dict = json.loads(synonym_file.read())

In [ ]:
synonym_word = wordOccurrence(synonym_dict)

100%|██████████| 150/150 [00:00<00:00, 16352.06it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurence_of_150_noun.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_word,jsonfile,ensure_ascii=False, indent=2)

# Word Occurrence for Verb

In [ ]:
import json
synonym_verb_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/100_verb_synonym.json', encoding='utf-8')
synonym_verb_dict = json.loads(synonym_verb_file.read())

In [ ]:
synonym_verb_word = wordOccurrence(synonym_verb_dict)

100%|██████████| 100/100 [00:00<00:00, 4867.31it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurence_of_100_verb.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_verb_word,jsonfile,ensure_ascii=False, indent=2)

# Word Occurrence for Adj

In [ ]:
import json
synonym_adj_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Synonym/50_adj_synonym.json', encoding='utf-8')
synonym_adj_dict = json.loads(synonym_adj_file.read())

In [ ]:
synonym_adj_word = wordOccurrence(synonym_adj_dict)

100%|██████████| 50/50 [00:00<00:00, 31151.99it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurence_of_50_adj.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_adj_word,jsonfile,ensure_ascii=False, indent=2)

# Function to show Word Occurrence and Example Sentence

In [ ]:
def get_num_occurence(item):
    # Return default value that is equivalent to no skill
    return item[1]['occurence']

In [ ]:
def wordOccurrenceSentence(synonym_dict):
  synonym_word_sentence = {}

  for key in tqdm(synonym_dict.keys()):
    word_occurence = {}
    for sub_key in synonym_dict[key]:
      for word in synonym_dict[key][sub_key]:
        if word in word_occurence:
          word_occurence[word]['occurence'] += 1
          if len(word_occurence[word]['sentence']) < 3:
            sentence = sub_key.replace(key, '<mask>', 1)
            word_occurence[word]['sentence'].append(sentence)
        else:
          sentence = sub_key.replace(key, '<mask>', 1)
          word_occurence[word] = {'occurence': 1, 'sentence': [sentence]}
    word_occurence = {k: v for k, v in sorted(word_occurence.items(), key=get_num_occurence, reverse=True)}

    synonym_word_sentence[key] = word_occurence

  return synonym_word_sentence

# Word Occurrence and Example Sentence for Noun

In [ ]:
synonym_word_sentence = wordOccurrenceSentence(synonym_dict)

100%|██████████| 150/150 [00:00<00:00, 5304.81it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurence_150_noun_sentence.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_word_sentence,jsonfile,ensure_ascii=False, indent=2)

# Word Occurrence and Example Sentence for Verb

In [ ]:
synonym_verb_sentence = wordOccurrenceSentence(synonym_verb_dict)

100%|██████████| 100/100 [00:00<00:00, 1934.07it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurence_100_verb_sentence.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_verb_sentence,jsonfile,ensure_ascii=False, indent=2)

# Word Occurrence and Example Sentence for Adj

In [ ]:
synonym_adj_sentence = wordOccurrenceSentence(synonym_adj_dict)

100%|██████████| 50/50 [00:00<00:00, 13055.79it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurrence_50_adj_sentence.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_adj_sentence,jsonfile,ensure_ascii=False, indent=2)

# BERT Concatenation Candidate

**Load Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
lm_model = RobertaForMaskedLM.from_pretrained('vinai/phobert-base')
raw_model = AutoModel.from_pretrained('vinai/phobert-base', output_hidden_states=True, output_attentions=True)

def load_transformers():
    return tokenizer, lm_model, raw_model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

**Function to find synonym candidate**

In [ ]:
def BertConcatCandidate(sentence, target_word):
  dict = {}
  res = []
  masked_sent = sentence.replace(target_word, tokenizer.mask_token, 1)

  #Get the input token IDs of the input consisting of: the original sentence + separator + the masked sentence.
  input_ids = tokenizer.encode(" "+sentence, " "+masked_sent, add_special_tokens=True)
  masked_position = input_ids.index(tokenizer.mask_token_id)

  original_output = raw_model(torch.tensor(input_ids).reshape(1, len(input_ids)))

  #Get the predictions of the Masked LM transformer.
  with torch.no_grad():
          output = lm_model(torch.tensor(input_ids).reshape(1, len(input_ids)))

  logits = output[0].squeeze()

  #Get top guesses: their token IDs, scores, and words.
  mask_logits = logits[masked_position].squeeze()
  top_tokens = torch.topk(mask_logits, k=50, dim=0)[1]
  scores = torch.softmax(mask_logits, dim=0)[top_tokens].tolist()
  words = [tokenizer.decode(i.item()).strip() for i in top_tokens]

  for i in range(len(words)):
    word = words[i]
    if '@' in word or word in string.punctuation or '<' in word or has_numbers(word):
      continue
    res.append(words[i].replace(' ', ''))

  dict['sentence'] = sentence
  dict['target_word'] = target_word
  dict['synonym_candidates'] = res

  return dict

In [ ]:
def findSynonymCandidate(data, keys):
  final_dict = {}

  for key in tqdm(keys):
    for sentence in data[key]:
      dict = BertConcatCandidate(sentence, key)
      if dict['target_word'] not in final_dict.keys():
        final_dict[dict['target_word']] = {dict['sentence']:dict['synonym_candidates']}
      else:
        final_dict[dict['target_word']][dict['sentence']] = dict['synonym_candidates']

  return final_dict

**Noun synonym candidate**

In [ ]:
noun_and_question_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/noun_and_question.json')
noun_and_question_data = json.load(noun_and_question_file)

In [ ]:
noun_candidate = findSynonymCandidate(noun_and_question_data, noun_keys)

100%|██████████| 150/150 [04:46<00:00,  1.91s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Candidate/noun_synonym_candidate.json","w", encoding='utf-8') as jsonfile:
    json.dump(noun_candidate,jsonfile,ensure_ascii=False, indent=2)

**Verb synonym candidate**

In [ ]:
verb_and_question_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/verb_and_question.json')
verb_and_question_data = json.load(verb_and_question_file)

In [ ]:
verb_candidate = findSynonymCandidate(verb_and_question_data, verb_keys)

100%|██████████| 100/100 [08:36<00:00,  5.16s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Candidate/verb_synonym_candidate.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_candidate,jsonfile,ensure_ascii=False, indent=2)

**Adj synonym candidate**

In [ ]:
adj_and_question_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/adj_and_question.json')
adj_and_question_data = json.load(adj_and_question_file)

In [ ]:
adj_candidate = findSynonymCandidate(adj_and_question_data, adj_keys)

100%|██████████| 50/50 [00:38<00:00,  1.30it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Candidate/adj_synonym_candidate.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_candidate,jsonfile,ensure_ascii=False, indent=2)

# BERT Concatenation Synonym

In [ ]:
def findSynonymBertConcat(pos_synonym_list):
  synonym_dict = {}

  batch_sentence = {
      'text1': [],
      'text2': []
  }

  word_and_prob = {
      'word': [],
      'sentence': []
  }

  batch_size = 0
  cnt = 0

  for key in tqdm(pos_synonym_list.keys()):
    small_synonym_dict = {}
    for sub_key in pos_synonym_list[key]:
      for i in range(len(pos_synonym_list[key][sub_key])):
        pos_synonym_word = pos_synonym_list[key][sub_key][i]
        if pos_synonym_word in punctuation_set:
          continue

        sentence = sub_key.replace(key, pos_synonym_word)

        if batch_size < 16:
          batch_sentence['text1'].append(sub_key)
          batch_sentence['text2'].append(sentence)
          word_and_prob['word'].append(pos_synonym_word)
          word_and_prob['sentence'].append(sub_key)

          batch_size += 1
          cnt += 1
        else:
          batch_size = 0
          list_similarity_score = infer1(batch_sentence, tokenizer)
          batch_sentence['text1'] = []
          batch_sentence['text2'] = []

          for k in range(len(list_similarity_score.tolist())):
            similarity_score = list_similarity_score.tolist()[k]
            if similarity_score == 1:
              if word_and_prob['sentence'][k] not in small_synonym_dict.keys():
                #if word_and_prob['word'][k] not in small_synonym_dict[word_and_prob['sentence'][k]].keys():
                  small_synonym_dict[word_and_prob['sentence'][k]] = [word_and_prob['word'][k]]
              else:
                small_synonym_dict[word_and_prob['sentence'][k]].append(word_and_prob['word'][k])

          word_and_prob['word'] = []
          word_and_prob['sentence'] = []

      if batch_size > 0:
        batch_size = 0
        list_similarity_score = infer1(batch_sentence, tokenizer)
        batch_sentence['text1'] = []
        batch_sentence['text2'] = []

        for k in range(len(list_similarity_score.tolist())):
          similarity_score = list_similarity_score.tolist()[k]
          if similarity_score == 1:
            if word_and_prob['sentence'][k] not in small_synonym_dict.keys():
              #if word_and_prob['word'][k] not in small_synonym_dict[word_and_prob['sentence'][k]].keys():
                small_synonym_dict[word_and_prob['sentence'][k]] = [word_and_prob['word'][k]]
            else:
              small_synonym_dict[word_and_prob['sentence'][k]].append(word_and_prob['word'][k])

        word_and_prob['word'] = []
        word_and_prob['sentence'] = []
      synonym_dict[key] = small_synonym_dict

  return synonym_dict

**Noun synonym**

In [ ]:
import json
bert_concat_noun_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Candidate/noun_synonym_candidate.json', encoding='utf-8')
bert_concat_noun_list = json.loads(bert_concat_noun_file.read())

In [ ]:
bert_concat_noun_synonym_dict = findSynonymBertConcat(bert_concat_noun_list)

100%|██████████| 150/150 [11:23<00:00,  4.56s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Synonym/150_noun_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_concat_noun_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

**Verb synonym**

In [ ]:
import json
bert_concat_verb_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Candidate/verb_synonym_candidate.json', encoding='utf-8')
bert_concat_verb_list = json.loads(bert_concat_verb_file.read())

In [ ]:
bert_concat_verb_synonym_dict = findSynonymBertConcat(bert_concat_verb_list)

100%|██████████| 100/100 [18:14<00:00, 10.95s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Synonym/100_verb_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_concat_verb_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

**Adj synonym**

In [ ]:
import json
bert_concat_adj_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Candidate/adj_synonym_candidate.json', encoding='utf-8')
bert_concat_adj_list = json.loads(bert_concat_adj_file.read())

In [ ]:
bert_concat_adj_synonym_dict = findSynonymBertConcat(bert_concat_adj_list)

100%|██████████| 50/50 [01:31<00:00,  1.83s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Synonym/50_adj_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_concat_adj_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

# BERT Concatenation Occurrence

**Noun Occurrence**

In [ ]:
import json
bert_concat_noun_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Synonym/150_noun_synonym.json', encoding='utf-8')
bert_concat_noun_synonym_dict = json.loads(bert_concat_noun_synonym_file.read())

In [ ]:
bert_concat_noun_occurrence = wordOccurrence(bert_concat_noun_synonym_dict)

100%|██████████| 150/150 [00:00<00:00, 15523.34it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_of_150_noun.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_concat_noun_occurrence,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
synonym_noun_sentence = wordOccurrenceSentence(bert_concat_noun_synonym_dict)

100%|██████████| 150/150 [00:00<00:00, 4495.66it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_150_noun_sentence.json.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_noun_sentence,jsonfile,ensure_ascii=False, indent=2)

**Verb Occurrence**

In [ ]:
import json
bert_concat_verb_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Synonym/100_verb_synonym.json', encoding='utf-8')
bert_concat_verb_synonym_dict = json.loads(bert_concat_verb_synonym_file.read())

In [ ]:
bert_concat_verb_occurrence = wordOccurrence(bert_concat_verb_synonym_dict)

100%|██████████| 100/100 [00:00<00:00, 3556.88it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_of_100_verb.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_concat_verb_occurrence,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
synonym_verb_sentence = wordOccurrenceSentence(bert_concat_verb_synonym_dict)

100%|██████████| 100/100 [00:00<00:00, 1794.34it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_100_verb_sentence.json.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_verb_sentence,jsonfile,ensure_ascii=False, indent=2)

**Adj Occurrence**

In [ ]:
import json
bert_concat_adj_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Synonym/50_adj_synonym.json', encoding='utf-8')
bert_concat_adj_synonym_dict = json.loads(bert_concat_adj_synonym_file.read())

In [ ]:
bert_concat_adj_occurrence = wordOccurrence(bert_concat_adj_synonym_dict)

100%|██████████| 50/50 [00:00<00:00, 17524.46it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_of_50_adj.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_concat_adj_occurrence,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
synonym_adj_sentence = wordOccurrenceSentence(bert_concat_adj_synonym_dict)

100%|██████████| 50/50 [00:00<00:00, 12329.66it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_50_adj_sentence.json.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_adj_sentence,jsonfile,ensure_ascii=False, indent=2)

# BERT Partial Mask Candidate

**Load Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
lm_model = RobertaForMaskedLM.from_pretrained('vinai/phobert-base')
raw_model = AutoModel.from_pretrained('vinai/phobert-base', output_hidden_states=True, output_attentions=True)

def load_transformers():
    return tokenizer, lm_model, raw_model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Calculate Score**

In [ ]:
#Calculates the similarity score
def similarity_score(original_output, subst_output, k):
    mask_idx = k
    cos_sim = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    weights = torch.div(torch.stack(list(original_output[3])).squeeze().sum(0).sum(0), (12 * 12.0))

    #Calculate the similarittimey score
    #SIM(x, x'; k) = sum_i^L [ w_{i,k} * cos(h(x_i|x), h(x_i'|x')) ]

    #subst_output = raw_model(sent.reshape(1, sent.shape[0]))
    suma = 0.0
    sent_len = original_output[2][2].shape[1]

    for token_idx in range(sent_len):
        original_hidden = original_output[2]
        subst_hidden = subst_output[2]

        #Calculate the contextualized representation of the i-th word as a concatenation of RoBERTa's values in its last four layers
        context_original = torch.cat( tuple( [original_hidden[hs_idx][:, token_idx, :] for hs_idx in [1, 2, 3, 4]] ), dim=1)
        context_subst = torch.cat( tuple( [subst_hidden[hs_idx][:, token_idx, :] for hs_idx in [1, 2, 3, 4]] ), dim=1)
        suma += weights[mask_idx][token_idx] * cos_sim(context_original, context_subst)

    substitute_validation = suma
    return substitute_validation


#Calculates the proposal score
def proposal_score(original_score, subst_scores):
    subst_scores = torch.tensor(subst_scores)
    return np.log( torch.div(subst_scores , (1.0 - original_score)) )


#Calculates the proposal scores, substitute validation scores, and then the final score for each candidate word's fit as a substitution.
def calc_scores(scr, sentences, original_output, original_score, mask_index):
    #Get representations of all substitute sentences
    _, _, raw_model = load_transformers()
    subst_output = raw_model(sentences)

    prop_score = proposal_score(original_score, scr)
    substitute_validation = similarity_score(original_output, subst_output, mask_index)

    alpha = 0.003
    final_score = substitute_validation + alpha*prop_score

    '''
    print("Proposal score: " + str(prop_score))
    print("Subst. validation: " + str(substitute_validation))
    print("Final score for " + str(final_score) + "\n")
    '''
    return final_score, prop_score, substitute_validation

**Check if candidate consists of number in it**

In [ ]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

**Function to find candidate**

In [ ]:
def bertPartialCandidate(sentence, target_word):
  original_tokenizer = tokenizer.encode(" "+sentence, return_tensors="pt")

  original_output = raw_model(original_tokenizer)

  inputs_embeds = original_output[2][1]

  target_token_id = tokenizer.encode(" "+target_word)[1]
  input_ids = tokenizer.encode(" "+sentence)
  mask_position = input_ids.index(target_token_id)

  embedding_dim = 768
  dropout_percent = 0.3
  dropout_amount = round(dropout_percent*embedding_dim)

  all_scores = dict()
  all_counts = dict()
  num_iterations = 5
  for it in range(num_iterations):
      #Choose the weight indices to drop out.
      dropout_indices = np.random.choice(embedding_dim, dropout_amount, replace=False)
      inputs_embeds[0, mask_position, dropout_indices] = 0

      #Pass the embeddings where masked word's embedding is partially droppped out to the model
      with torch.no_grad():
              output = lm_model(inputs_embeds=inputs_embeds)
      logits = output[0].squeeze()

      #Get top guesses
      mask_logits = logits[mask_position]
      top_tokens = torch.topk(mask_logits, k=50, dim=0)[1]
      scores = torch.softmax(mask_logits, dim=0)[top_tokens].tolist()
      words = [tokenizer.decode(i.item()).strip() for i in top_tokens]

      original_score = torch.softmax(mask_logits, dim=0)[target_token_id]
      sentences = list()

      for i in range(len(words)):
          subst_word = top_tokens[i]
          input_ids[mask_position] = int(subst_word)
          sentences.append(list(input_ids))

      sentences = torch.tensor(sentences)

      finals, props, subval = calc_scores(scores, sentences, original_output, original_score, mask_position)
      finals = map(lambda f : float(f), finals)
      props = map(lambda f : float(f), props)
      subval = map(lambda f : float(f), subval)

      #Update total scores and counts in the dictionary
      res = dict(zip(words, finals))
      for w, s in res.items():
          all_scores[w] = all_scores[w] + s if w in all_scores.keys() else s
          all_counts[w] = all_counts[w] + 1 if w in all_counts.keys() else 1

  #Get the average of accumulated scores.
  for w, s in all_scores.items():
      all_scores[w] = s / all_counts[w]
  words, finals = list(all_scores.keys()), list(all_scores.values())


  #Sort the found substitutes by scores and print them out.
  x = dict(zip(words, finals))
  finish = list(sorted(x.items(), key=lambda item: item[1], reverse=True))[:50]

  res = {}
  finalDict = {}
  for i in range(len(finish)):
    if '@' in finish[i][0] or finish[i][0] in string.punctuation or '<' in finish[i][0] or has_numbers(finish[i][0]) or target_word == finish[i][0].replace(' ', ''):
      continue
    res[finish[i][0].replace(' ', '')] = finish[i][1]

  finalDict['sentence'] = sentence
  finalDict['target_word'] = target_word
  finalDict['synonym_candidates'] = res

  return finalDict

In [ ]:
def findSynonymCandidate(data, keys):
  final_dict = {}

  for key in tqdm(keys):
    for sentence in data[key]:
      dict = bertPartialCandidate(sentence, key)
      if dict['target_word'] not in final_dict.keys():
        final_dict[dict['target_word']] = {dict['sentence']:dict['synonym_candidates']}
      else:
        final_dict[dict['target_word']][dict['sentence']] = dict['synonym_candidates']

  return final_dict

**Noun synonym candidate**

In [ ]:
noun_and_question_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/noun_and_question.json')
noun_and_question_data = json.load(noun_and_question_file)

In [ ]:
noun_candidate = findSynonymCandidate(noun_and_question_data, noun_keys)

100%|██████████| 150/150 [1:24:36<00:00, 33.84s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Candidate/noun_synonym_candidate.json","w", encoding='utf-8') as jsonfile:
    json.dump(noun_candidate,jsonfile,ensure_ascii=False, indent=2)

**Verb synonym candidate**

In [ ]:
verb_and_question_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/verb_and_question.json')
verb_and_question_data = json.load(verb_and_question_file)

In [ ]:
verb_candidate = findSynonymCandidate(verb_and_question_data, verb_keys)

100%|██████████| 100/100 [2:24:05<00:00, 86.45s/it] 


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Candidate/verb_synonym_candidate.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_candidate,jsonfile,ensure_ascii=False, indent=2)

**Adj synonym candidate**

In [ ]:
adj_and_question_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Extracted Words and Questions/adj_and_question.json')
adj_and_question_data = json.load(adj_and_question_file)

In [ ]:
adj_candidate = findSynonymCandidate(adj_and_question_data, adj_keys)

100%|██████████| 50/50 [09:41<00:00, 11.62s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Candidate/adj_synonym_candidate.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_candidate,jsonfile,ensure_ascii=False, indent=2)

# BERT Partial Synonym

In [ ]:
def findSynonymBertPartial(pos_synonym_list):
  synonym_dict = {}

  batch_sentence = {
      'text1': [],
      'text2': []
  }

  word_and_prob = {
      'word': [],
      'sentence': []
  }

  batch_size = 0
  cnt = 0

  for key in tqdm(pos_synonym_list.keys()):
    small_synonym_dict = {}
    for sub_key in pos_synonym_list[key]:
      listWord = list(pos_synonym_list[key][sub_key].keys())
      for i in range(len(listWord)):
        pos_synonym_word = listWord[i]

        sentence = sub_key.replace(key, pos_synonym_word)

        if batch_size < 16:
          batch_sentence['text1'].append(sub_key)
          batch_sentence['text2'].append(sentence)
          word_and_prob['word'].append(pos_synonym_word)
          word_and_prob['sentence'].append(sub_key)

          batch_size += 1
          cnt += 1
        else:
          batch_size = 0
          list_similarity_score = infer1(batch_sentence, tokenizer)
          batch_sentence['text1'] = []
          batch_sentence['text2'] = []

          for k in range(len(list_similarity_score.tolist())):
            similarity_score = list_similarity_score.tolist()[k]
            if similarity_score == 1:
              if word_and_prob['sentence'][k] not in small_synonym_dict.keys():
                small_synonym_dict[word_and_prob['sentence'][k]] = [word_and_prob['word'][k]]
              else:
                small_synonym_dict[word_and_prob['sentence'][k]].append(word_and_prob['word'][k])

          word_and_prob['word'] = []
          word_and_prob['sentence'] = []

      if batch_size > 0:
        batch_size = 0
        list_similarity_score = infer1(batch_sentence, tokenizer)
        batch_sentence['text1'] = []
        batch_sentence['text2'] = []

        for k in range(len(list_similarity_score.tolist())):
          similarity_score = list_similarity_score.tolist()[k]
          if similarity_score == 1:
            if word_and_prob['sentence'][k] not in small_synonym_dict.keys():
              small_synonym_dict[word_and_prob['sentence'][k]] = [word_and_prob['word'][k]]
            else:
              small_synonym_dict[word_and_prob['sentence'][k]].append(word_and_prob['word'][k])

        word_and_prob['word'] = []
        word_and_prob['sentence'] = []
      synonym_dict[key] = small_synonym_dict

  return synonym_dict

**Noun Synonym**

In [ ]:
import json
bert_partial_noun_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Candidate/noun_synonym_candidate.json', encoding='utf-8')
bert_partial_noun_list = json.loads(bert_partial_noun_file.read())

In [ ]:
bert_partial_noun_synonym_dict = findSynonymBertPartial(bert_partial_noun_list)

100%|██████████| 150/150 [10:06<00:00,  4.04s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Synonym/150_noun_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_partial_noun_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

**Verb Synonym**

In [ ]:
import json
bert_partial_verb_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Candidate/verb_synonym_candidate.json', encoding='utf-8')
bert_partial_verb_list = json.loads(bert_partial_verb_file.read())

In [ ]:
bert_partial_verb_synonym_dict = findSynonymBertPartial(bert_partial_verb_list)

100%|██████████| 100/100 [15:40<00:00,  9.40s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Synonym/100_verb_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_partial_verb_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

**Adj Synonym**

In [ ]:
import json
bert_partial_adj_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Candidate/adj_synonym_candidate.json', encoding='utf-8')
bert_partial_adj_list = json.loads(bert_partial_adj_file.read())

In [ ]:
bert_partial_adj_synonym_dict = findSynonymBertPartial(bert_partial_adj_list)

100%|██████████| 50/50 [01:18<00:00,  1.57s/it]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Synonym/50_adj_synonym.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_partial_adj_synonym_dict,jsonfile,ensure_ascii=False, indent=2)

# BERT Partial Occurrence

**Noun**

In [ ]:
import json
bert_partial_noun_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Synonym/150_noun_synonym.json', encoding='utf-8')
bert_partial_noun_synonym_dict = json.loads(bert_partial_noun_synonym_file.read())

In [ ]:
bert_partial_noun_occurrence = wordOccurrence(bert_partial_noun_synonym_dict)

100%|██████████| 150/150 [00:00<00:00, 13365.60it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_of_150_noun.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_partial_noun_occurrence,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
synonym_noun_sentence = wordOccurrenceSentence(bert_partial_noun_synonym_dict)

100%|██████████| 150/150 [00:00<00:00, 4555.59it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_150_noun_sentence.json.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_noun_sentence,jsonfile,ensure_ascii=False, indent=2)

**Verb**

In [ ]:
import json
bert_partial_verb_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Synonym/100_verb_synonym.json', encoding='utf-8')
bert_partial_verb_synonym_dict = json.loads(bert_partial_verb_synonym_file.read())

In [ ]:
bert_partial_verb_occurrence = wordOccurrence(bert_partial_verb_synonym_dict)

100%|██████████| 100/100 [00:00<00:00, 3980.25it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_of_100_verb.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_partial_verb_occurrence,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
synonym_verb_sentence = wordOccurrenceSentence(bert_partial_verb_synonym_dict)

100%|██████████| 100/100 [00:00<00:00, 2111.73it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_100_verb_sentence.json.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_verb_sentence,jsonfile,ensure_ascii=False, indent=2)

**Adj**

In [ ]:
import json
bert_partial_adj_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Synonym/50_adj_synonym.json', encoding='utf-8')
bert_partial_adj_synonym_dict = json.loads(bert_partial_adj_synonym_file.read())

In [ ]:
bert_partial_adj_occurrence = wordOccurrence(bert_partial_adj_synonym_dict)

100%|██████████| 50/50 [00:00<00:00, 28513.28it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_of_50_adj.json","w", encoding='utf-8') as jsonfile:
    json.dump(bert_partial_adj_occurrence,jsonfile,ensure_ascii=False, indent=2)

In [ ]:
synonym_adj_sentence = wordOccurrenceSentence(bert_partial_adj_synonym_dict)

100%|██████████| 50/50 [00:00<00:00, 8493.59it/s]


In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_50_adj_sentence.json.json","w", encoding='utf-8') as jsonfile:
    json.dump(synonym_adj_sentence,jsonfile,ensure_ascii=False, indent=2)

# Function to join BERT to Wordnet

In [ ]:
def joinBertBaseline(bert_data, wordnet_data):
  res = {}
  for key in wordnet_data.keys():
    tokenzied_key = key.replace(' ', '_')
    if tokenzied_key in bert_data.keys():
      res[key] = {"wordnet": wordnet_data[key], "bert": bert_data[tokenzied_key]}
  return res

# Load Wordnet Data

In [ ]:
noun_wordnet_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/wordnet_noun_synonym.json')
noun_wordnet_data = json.load(noun_wordnet_file)

In [ ]:
verb_wordnet_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/wordnet_verb_synonym.json')
verb_wordnet_data = json.load(verb_wordnet_file)

In [ ]:
adj_wordnet_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet Synonym/wordnet_adj_synonym.json')
adj_wordnet_data = json.load(adj_wordnet_file)

# Wordnet and Bert Baseline

**Noun**

In [ ]:
noun_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurrence_150_noun_sentence.json')
bert_noun_data = json.load(noun_data_file)

In [ ]:
noun_bert_wordnet = joinBertBaseline(bert_noun_data, noun_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Baseline/noun_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(noun_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

**Verb**

In [ ]:
verb_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurrence_100_verb_sentence.json')
bert_verb_data = json.load(verb_data_file)

In [ ]:
verb_bert_wordnet = joinBertBaseline(bert_verb_data, verb_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Baseline/verb_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

**Adj**

In [ ]:
adj_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Full Mask/Occurrence/occurrence_50_adj_sentence.json')
bert_adj_data = json.load(adj_data_file)

In [ ]:
adj_bert_wordnet = joinBertBaseline(bert_adj_data, adj_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Baseline/adj_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

# Wordnet and BERT Concatenation

**Noun**

In [ ]:
noun_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_150_noun_sentence.json.json')
bert_noun_data = json.load(noun_data_file)

In [ ]:
noun_bert_wordnet = joinBertBaseline(bert_noun_data, noun_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/noun_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(noun_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

**Verb**

In [ ]:
verb_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_100_verb_sentence.json.json')
bert_verb_data = json.load(verb_data_file)

In [ ]:
verb_bert_wordnet = joinBertBaseline(bert_verb_data, verb_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/verb_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

**Adj**

In [ ]:
adj_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Concatenation/Occurrence/occurrence_50_adj_sentence.json.json')
bert_adj_data = json.load(adj_data_file)

In [ ]:
adj_bert_wordnet = joinBertBaseline(bert_adj_data, adj_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/adj_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

# Wordnet and Bert Partial Mask

**Noun**

In [ ]:
noun_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_150_noun_sentence.json.json')
bert_noun_data = json.load(noun_data_file)

In [ ]:
noun_bert_wordnet = joinBertBaseline(bert_noun_data, noun_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Partial Mask/noun_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(noun_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

**Verb**

In [ ]:
verb_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_100_verb_sentence.json.json')
bert_verb_data = json.load(verb_data_file)

In [ ]:
verb_bert_wordnet = joinBertBaseline(bert_verb_data, verb_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Partial Mask/verb_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(verb_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

**Adj**

In [ ]:
adj_data_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Bert Partial Mask/Occurrence/occurrence_50_adj_sentence.json.json')
bert_adj_data = json.load(adj_data_file)

In [ ]:
adj_bert_wordnet = joinBertBaseline(bert_adj_data, adj_wordnet_data)

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Partial Mask/adj_bert_wordnet.json","w", encoding='utf-8') as jsonfile:
    json.dump(adj_bert_wordnet,jsonfile,ensure_ascii=False, indent=2)

# Write 300 words to CSV

In [ ]:
import csv
cnt=0
header = ['word_type', 'word', 'is_good']

with open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/300_words.csv', 'w') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(header)

    for word in all_300_words:
    # write a row to the csv file
      if cnt < 150:
        writer.writerow(['noun', word, ''])
      elif cnt > 149 and cnt < 250:
        writer.writerow(['verb', word, ''])
      else:
        writer.writerow(['adj', word, ''])
      cnt+=1

# Function to write synonym to CSV

In [ ]:
import csv

def writeSynonymToCSV(path):
  cnt=0
  header = ['word_type', 'word', 'wordnet_synonym', 'bert_synonym', 'is_good']

  with open(path, 'w') as f:
      # create the csv writer
      writer = csv.writer(f)
      writer.writerow(header)

      for word in all_300_words:
      # write a row to the csv file
        detokenized_word = word.replace('_', ' ')
        if cnt < 150:
          writer.writerow(['noun', word, noun_synonym_data[detokenized_word]['wordnet'], list(noun_synonym_data[detokenized_word]['bert'].keys()), ''])
        elif cnt > 149 and cnt < 250:
          writer.writerow(['verb', word, verb_synonym_data[detokenized_word]['wordnet'], list(verb_synonym_data[detokenized_word]['bert'].keys()), ''])
        else:
          writer.writerow(['adj', word, adj_synonym_data[detokenized_word]['wordnet'], list(adj_synonym_data[detokenized_word]['bert'].keys()), ''])
        cnt+=1

# Write 300 words and synonym to CSV (BERT Concatenation)

In [ ]:
noun_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/noun_bert_wordnet.json')
noun_synonym_data = json.load(noun_synonym_file)

verb_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/verb_bert_wordnet.json')
verb_synonym_data = json.load(verb_synonym_file)

adj_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/adj_bert_wordnet.json')
adj_synonym_data = json.load(adj_synonym_file)

In [ ]:
writeSynonymToCSV('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/synonym.csv')

# Write 300 words and synonym to CSV (BERT Partial Mask)

In [ ]:
noun_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Partial Mask/noun_bert_wordnet.json')
noun_synonym_data = json.load(noun_synonym_file)

verb_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Partial Mask/verb_bert_wordnet.json')
verb_synonym_data = json.load(verb_synonym_file)

adj_synonym_file = open('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Partial Mask/adj_bert_wordnet.json')
adj_synonym_data = json.load(adj_synonym_file)

In [ ]:
writeSynonymToCSV('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Partial Mask/synonym.csv')

# Function create CSV to be revised

In [ ]:
import csv

def synonymToBeRevised(path):
  header = ['Từ', 'Từ thay thế', 'Ngữ cảnh (chỗ <mask> là vị trí của từ/từ thay thế)', 'Phân loại', 'Chú thích (nếu cần thiết)']

  with open(path, 'w') as f:
      # create the csv writer
      writer = csv.writer(f)
      writer.writerow(header)

      for dict in [noun_synonym_data, verb_synonym_data, adj_synonym_data]:
        for key in dict.keys():
          cnt = 0
          tmp = []
          for synonym_word in dict[key]['bert'].keys():
            detokenized_synonym = synonym_word.replace('_', ' ')
            if key.lower() != detokenized_synonym.lower() and cnt < 11 and detokenized_synonym.lower() not in tmp:
              writer.writerow([key, synonym_word, dict[key]['bert'][synonym_word]['sentence'], '', ''])
              tmp.append(detokenized_synonym.lower())
              cnt += 1

In [ ]:
synonymToBeRevised('/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/Wordnet and Bert Concatenation/synonymToBeRevised.csv')

# Word2Vec Baseline



In [ ]:
from gensim.models import KeyedVectors
from gensim import models

word2vec_path = '/content/drive/MyDrive/Final Project Zalo Internship/Final Project/baomoi.window2.vn.model.bin.gz'
w2v_model = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
w2v_dict = {}
for wrd in tqdm(word):
  if wrd in w2v_model.vocab:
    w2v_dict[wrd] = w2v_model.most_similar(wrd.lower(), topn=50)

100%|██████████| 1205/1205 [00:24<00:00, 48.63it/s]


In [ ]:
synonym_Occurrence_dict['khu_vực']

{'khu_vực': {'occurence': 186,
  'sentence': ['tổng_sản_phẩm nội_địa của <mask> Mỹ Latinh năm 2010 là bao_nhiêu',
   'Tây_Ban_Nha nằm ở <mask> nào',
   'Liên_Xô nằm ở <mask> nào']},
 'vùng': {'occurence': 180,
  'sentence': ['tổng_sản_phẩm nội_địa của <mask> Mỹ Latinh năm 2010 là bao_nhiêu',
   'Tây_Ban_Nha nằm ở <mask> nào',
   'Liên_Xô nằm ở <mask> nào']},
 'khu': {'occurence': 135,
  'sentence': ['tổng_sản_phẩm nội_địa của <mask> Mỹ Latinh năm 2010 là bao_nhiêu',
   'Tây_Ban_Nha nằm ở <mask> nào',
   'Triều_Tiên nằm ở <mask> nào']},
 'tỉnh': {'occurence': 125,
  'sentence': ['Tây_Ban_Nha nằm ở <mask> nào',
   'Liên_Xô nằm ở <mask> nào',
   'Macedonia thuộc <mask> nào']},
 'lãnh_thổ': {'occurence': 101,
  'sentence': ['tổng_sản_phẩm nội_địa của <mask> Mỹ Latinh năm 2010 là bao_nhiêu',
   'Tây_Ban_Nha nằm ở <mask> nào',
   'Liên_Xô nằm ở <mask> nào']},
 'vùng_đất': {'occurence': 84,
  'sentence': ['Tây_Ban_Nha nằm ở <mask> nào',
   'Liên_Xô nằm ở <mask> nào',
   'Canada nằm ở <mask> n

In [ ]:
w2v_dict['bảng_tuần_hoàn']

[('mendeleev', 0.6717042326927185),
 ('tác_nhân_vật_lý', 0.6478486657142639),
 ('hỗn_hợp_chất', 0.6426305174827576),
 ('vũ_khí_sinh', 0.6344994306564331),
 ('octinoxate', 0.6315604448318481),
 ('vanillin', 0.6315262317657471),
 ('avobenzone', 0.6307601928710938),
 ('axiton', 0.6129394769668579),
 ('Độc_chất', 0.6077231168746948),
 ('khoa_kiểm_nghiệm', 0.5963605642318726),
 ('nhà_địa', 0.5916277170181274),
 ('hoá_học', 0.5723886489868164),
 ('ununseptium', 0.5701664686203003),
 ('berkelium', 0.5693877935409546),
 ('baff', 0.5643936991691589),
 ('monell', 0.5609896183013916),
 ('livermorium', 0.559063732624054),
 ('hợp_chất_khí', 0.552109956741333),
 ('ngộ_độc_chất', 0.5512978434562683),
 ('ubiquinone', 0.5468034744262695),
 ('hóa_học', 0.5415852665901184),
 ('phthalates', 0.5385777354240417),
 ('silicat', 0.5336449146270752),
 ('rohm', 0.5315697193145752),
 ('cấm_vũ_khí', 0.5305625200271606),
 ('nguyễn_khánh_diệu_hồng', 0.5303667783737183),
 ('perfluorinated', 0.5268809795379639),
 ('số

In [ ]:
with open("/content/drive/MyDrive/Final Project Zalo Internship/Final Project/Categorized vocab (No Trec57)/word2vec.json","w", encoding='utf-8') as jsonfile:
    json.dump(w2v_dict,jsonfile,ensure_ascii=False, indent=2)